<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Portugués a Ingles: ST Baseline Whisper Small**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/MyDrive/Colab Notebooks/TA/A3/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import whisper
import jiwer
from whisper.normalizers import BasicTextNormalizer

from tqdm.notebook import tqdm
import pandas as pd

model = whisper.load_model("small")

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [4]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/TA/A3/pt"
from datasets import load_dataset

raw_datasets = load_dataset("facebook/covost2", 'pt_en', data_dir=output_dir)

print(raw_datasets)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 9158
    })
    validation: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 3318
    })
    test: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 4023
    })
})


In [5]:
data = raw_datasets["test"][:1000]

In [6]:
translations = []
references = []
sources = []

for i in tqdm(range(1000)):
    audio_path = data["file"][i]
    source_sentence = data["sentence"][i]
    reference_translation = data["translation"][i]

    references.append(reference_translation)
    sources.append(source_sentence)

    transcription = model.transcribe(audio_path, language="Portuguese", task="translate")['text']

    translations.append(transcription)

print(f"Processed {len(translations)} audio clips.")

  0%|          | 0/1000 [00:00<?, ?it/s]

Processed 1000 audio clips.


In [7]:
data = pd.DataFrame(dict(translation=translations, reference=references, source=sources))
pd.set_option('display.max_colwidth', None)
data.head(5)

,translation,reference,source
0,Paying costs to WHO,Borrow money from people in the village,Pedir dinheiro emprestado às pessoas da aldeia
1,We broke up.,Lock them up,Trancá-los
2,YouTube is still the best video platform,Youtube is still the best video platform.,O Youtube ainda é a melhor plataforma de vídeos.
3,Girl and boy kissing in the shadows.,A girl and a boy kissing in the shadows,Menina e menino beijando nas sombras
4,"Particularly, my cut couscous I like to eat well instead of greggado.","I actually prefer to eat my sliced couscous as it is, instead of grilled.","Particularmente, meu cuscuz cortado eu gosto de comer bem, em vez de grelhado."


Automatic translations, references and sources are normalized using the Whisper basic text standardisation/normalization module

In [8]:
normalizer = BasicTextNormalizer()

data["translation_clean"] = [normalizer(text) for text in data["translation"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data["source_clean"] = [normalizer(text) for text in data["source"]]
data.head(5)

,translation,reference,source,translation_clean,reference_clean,source_clean
0,Paying costs to WHO,Borrow money from people in the village,Pedir dinheiro emprestado às pessoas da aldeia,paying costs to who,borrow money from people in the village,pedir dinheiro emprestado às pessoas da aldeia
1,We broke up.,Lock them up,Trancá-los,we broke up,lock them up,trancá los
2,YouTube is still the best video platform,Youtube is still the best video platform.,O Youtube ainda é a melhor plataforma de vídeos.,youtube is still the best video platform,youtube is still the best video platform,o youtube ainda é a melhor plataforma de vídeos
3,Girl and boy kissing in the shadows.,A girl and a boy kissing in the shadows,Menina e menino beijando nas sombras,girl and boy kissing in the shadows,a girl and a boy kissing in the shadows,menina e menino beijando nas sombras
4,"Particularly, my cut couscous I like to eat well instead of greggado.","I actually prefer to eat my sliced couscous as it is, instead of grilled.","Particularmente, meu cuscuz cortado eu gosto de comer bem, em vez de grelhado.",particularly my cut couscous i like to eat well instead of greggado,i actually prefer to eat my sliced couscous as it is instead of grilled,particularmente meu cuscuz cortado eu gosto de comer bem em vez de grelhado


<p style="page-break-after:always;"></p>

In [9]:
!pip install -q evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.7 MB/s eta 0:00:00


In [10]:
from evaluate import load

metric = load("sacrebleu")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
result = metric.compute(predictions=data["translation_clean"], references=data["reference_clean"])
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 41.0


In [12]:
!pip install unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 28.1 MB/s eta 0:00:00


In [13]:
from evaluate import load
comet_metric = load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [14]:
comet_score = comet_metric.compute(predictions=data["translation_clean"], references=data["reference_clean"], sources=data["source_clean"])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [15]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 79.15 %


In [16]:
data.to_csv('/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ST_Small_test.csv', encoding='utf-8')